In [21]:
!pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.1 MB 6.3 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.1 MB 4.8 MB/s eta 0:00:02
   --------- ------------------------------ 2.6/11.1 MB 4.4 MB/s eta 0:00:02
   ------------- -------------------------- 3.7/11.1 MB 4.3 MB/s eta 0:00:02
   ---------------- ----------------------- 4.5/11.1 MB 4.2 MB/s eta 0:00:02
   ------------------ --------------------- 5.2/11.1 MB 4.1 MB/s eta 0:00:02
   --------------------- ------------------ 6.0/11.1 MB 4.1 MB/s eta 0:00:02
   ------------------------ --------------- 6.8/11.1 MB 4.1 MB/s eta 0:00:02
   --------------------------- ------------ 7.6/11.1 MB 4.0 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.1 MB 4.0 MB/s eta 0:00:01
   --------------------------------- ------ 9.4/11.1 MB 4.0 MB/s eta 0:00:01
   ------------------------------------ --- 10.2/11.1 MB 4.0 MB/s eta 0:00:01
   --

In [22]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split

# Load train.csv
df = pd.read_csv("Train.csv")

# Define paths for the new dataset structure
dataset_path = "datasets/GTSRB"
images_path = os.path.join(dataset_path, "images")
labels_path = os.path.join(dataset_path, "labels")

train_images_path = os.path.join(images_path, "train")
val_images_path = os.path.join(images_path, "val")
train_labels_path = os.path.join(labels_path, "train")
val_labels_path = os.path.join(labels_path, "val")

# Create required directories
for path in [train_images_path, val_images_path, train_labels_path, val_labels_path]:
    os.makedirs(path, exist_ok=True)

# Convert CSV annotations to YOLO format
def convert_to_yolo(row):
    img_width, img_height = row["Width"], row["Height"]
    x1, y1, x2, y2 = row["Roi.X1"], row["Roi.Y1"], row["Roi.X2"], row["Roi.Y2"]
    class_id = row["ClassId"]

    # Convert to YOLO format
    x_center = (x1 + x2) / 2 / img_width
    y_center = (y1 + y2) / 2 / img_height
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height

    return f"{class_id} {x_center:.3f} {y_center:.3f} {width:.3f} {height:.3f}"

# Split dataset into 80% train and 20% validation
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Function to process images and labels
def process_data(data, img_dest, label_dest):
    for _, row in data.iterrows():
        img_path = row["Path"]
        yolo_annotation = convert_to_yolo(row)

        # Copy image to new directory
        img_name = os.path.basename(img_path)
        new_img_path = os.path.join(img_dest, img_name)
        shutil.copy(img_path, new_img_path)

        # Create label file
        label_name = img_name.replace(".png", ".txt")
        new_label_path = os.path.join(label_dest, label_name)
        with open(new_label_path, "w") as f:
            f.write(yolo_annotation)

# Process training and validation datasets
process_data(train_df, train_images_path, train_labels_path)
process_data(val_df, val_images_path, val_labels_path)

print("✅ Dataset successfully prepared for YOLOv5 training!")


✅ Dataset successfully prepared for YOLOv5 training!


In [24]:
os.chdir("../..")
print(os.getcwd())

c:\Users\goush\Desktop\Programming\Django\SignSense
